### 线性规划

**线性规划的标准形式**
$$
\begin{align}
& min\:c^T x\\\
& Ax <= b\\\
& Aeq \cdot x = beq\\\
& lb \leq x \geq ub\\\
\end{align}
$$

**例题.1**
$$
\begin{align}
& max\: z = 2x_1 + 3x_2 - 5x_3 \\\
& x_1 + x_2 + x_3 = 7 \\\
& 2x_1 - 5x_2 + x_3 \geq 10 \\\
& x_1 + 3x_2 + x_3 \leq 12 \\\
& x_1,\,x_2,\,x_3 \leq 0 \\\
\end{align}
$$

In [8]:
import numpy as np
from scipy import optimize

z = np.array([-2, -3, 5])

Aeq = np.array([[1, 1, 1]])
Beq = np.array([7])

A = np.array([[-2, 5, -1], [1, 3, 1]])
B = np.array([-10, 12])

x1_bound = x2_bound = x3_bound = (0, None)

res = optimize.linprog(z, A_eq=Aeq, b_eq=Beq, A_ub=A, b_ub=B, bounds=(x1_bound, x2_bound, x3_bound))
res

     fun: -14.571428571428571
 message: 'Optimization terminated successfully.'
     nit: 2
   slack: array([3.85714286, 0.        ])
  status: 0
 success: True
       x: array([6.42857143, 0.57142857, 0.        ])

**例题.2 指派问题**

**n个人去干n项工作，每人干且仅干一项，若分配第 i人去干第j项工作，需要花费$c_{ij}$单位时间**

**问如何分配工作使得总时间最小**

In [10]:
import numpy as np
from scipy.optimize import linear_sum_assignment

efficiency_matrix = np.array([
    [12,7,9,7,9],
    [8,9,6,6,6],
    [7,17,12,14,12],
    [15,14,6,6,10],
    [4,10,7,10,6]
])

row_index, col_index=linear_sum_assignment(efficiency_matrix)
print(row_index+1) #行
print(col_index+1) #列
print(efficiency_matrix[row_index,col_index])

[1 2 3 4 5]
[2 3 1 4 5]
[7 6 7 6 6]
